# Movie ratings - Bayesian vaerage

In [1]:
import random
import pandas as pd

In [2]:
df = pd.read_csv("./ml-latest-small/ratings.csv")
ndf = pd.read_csv("./ml-latest-small/movies.csv")

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Ranking movies on arithmetic mean (average)

In [4]:
all_aa = df.groupby('movieId')["rating"].mean().sort_values(ascending=False)

## Ranking movies on Bayesian average

In [5]:
avg_rating = df['rating'].mean()

In [6]:
avg_count_rating = df.groupby("movieId")['rating'].count().mean()

In [7]:
count_ratings = df.groupby("movieId").count()['rating']

In [8]:
# del count_ratings[175981]

In [9]:
adf = pd.DataFrame(count_ratings, columns=['rating'])

In [10]:
adf.rename(columns = {'rating':'ratings_count'}, inplace = True)

In [11]:
adf['avg_rating'] = df.groupby('movieId').mean()['rating']

In [12]:
adf['overall_avg_rating'] = avg_rating

In [13]:
adf['overall_avg_count_rating'] = avg_count_rating

In [14]:
adf['w'] = adf['ratings_count']/(adf['overall_avg_count_rating'] + adf['ratings_count'])

In [15]:
adf['aa'] = all_aa

In [16]:
adf['ba'] = adf['w'] * adf['avg_rating'] + (1-adf['w']) * adf['overall_avg_rating']

In [17]:
adf.head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
1,215,3.920930,3.501557,10.369807,0.953988,3.920930,3.901634
2,110,3.431818,3.501557,10.369807,0.913850,3.431818,3.437826
3,52,3.259615,3.501557,10.369807,0.833737,3.259615,3.299841
4,7,2.357143,3.501557,10.369807,0.402998,2.357143,3.040360
5,49,3.071429,3.501557,10.369807,0.825335,3.071429,3.146557


In [18]:
adf.sort_values('aa', ascending=False).head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
88448,1,5.0,3.501557,10.369807,0.087952,5.0,3.633348
100556,1,5.0,3.501557,10.369807,0.087952,5.0,3.633348
143031,1,5.0,3.501557,10.369807,0.087952,5.0,3.633348
143511,1,5.0,3.501557,10.369807,0.087952,5.0,3.633348
143559,1,5.0,3.501557,10.369807,0.087952,5.0,3.633348


In [19]:
adf.sort_values('ba', ascending=False).head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
318,317,4.429022,3.501557,10.369807,0.968324,4.429022,4.399644
858,192,4.289062,3.501557,10.369807,0.948758,4.289062,4.248709
2959,218,4.272936,3.501557,10.369807,0.954592,4.272936,4.237909
1221,129,4.259690,3.501557,10.369807,0.925595,4.259690,4.203281
50,204,4.237745,3.501557,10.369807,0.951627,4.237745,4.202133


In [20]:
adf['diff'] = adf['aa'] - adf['ba']

In [21]:
candidates = adf[(adf['ratings_count'] > 30)].sort_values(['diff'], ascending=False)[:50]

In [22]:
ndf[ndf.movieId.isin(list(candidates.index))][['movieId', 'title']]

,movieId,title
28,29,"City of Lost Children, The (Cité des enfants p..."
52,58,"Postman, The (Postino, Il) (1994)"
228,265,Like Water for Chocolate (Como agua para choco...
681,899,Singin' in the Rain (1952)
690,908,North by Northwest (1959)
695,913,"Maltese Falcon, The (1941)"
696,914,My Fair Lady (1964)
742,969,"African Queen, The (1951)"
824,1084,Bonnie and Clyde (1967)
878,1172,Cinema Paradiso (Nuovo cinema Paradiso) (1989)


In [23]:
ndf[ndf.movieId == 119145].title.values[0]

'Kingsman: The Secret Service (2015)'

In [24]:
adf[adf.index == 119145]
adf.aa.rank(pct=True)[119145],adf.ba.rank(pct=True)[119145]

(0.7656314273961333, 0.9744960921431509)

In [25]:
def all_positions(movieId):
    df = pd.read_csv("./ml-latest-small/ratings.csv")
    df_ratings_for_movie = df[df.movieId == movieId].sort_values('timestamp', ascending=False)
    df = df.drop(df[df.movieId == movieId].index)
    
    pos = []
    
    items_proc = 0
    count = 1
    itr = df_ratings_for_movie.itertuples()
    not_done = True
    while not_done:
        
        tuples = []
        for _ in range(count):
            try:
                t = next(itr)
                tuples.append(t)
            except StopIteration:
                not_done = False
                break
        
        items_proc += len(tuples)
        
        df = df.append(tuples, ignore_index=True)
        all_aa = df.groupby('movieId')["rating"].mean().sort_values(ascending=False)
        avg_rating = df['rating'].mean()
        avg_count_rating = df.groupby("movieId")['rating'].count().mean()
        count_ratings = df.groupby("movieId").count()['rating']
        adf = pd.DataFrame(count_ratings, columns=['rating'])
        adf.rename(columns = {'rating':'ratings_count'}, inplace = True)
        adf['avg_rating'] = df.groupby('movieId').mean()['rating']
        adf['overall_avg_rating'] = avg_rating
        adf['overall_avg_count_rating'] = avg_count_rating
        adf['w'] = adf['ratings_count']/(adf['overall_avg_count_rating'] + adf['ratings_count'])
        adf['aa'] = all_aa
        adf['ba'] = adf['w'] * adf['avg_rating'] + (1-adf['w']) * adf['overall_avg_rating']
        adf['ba_rank'] = adf.ba.rank(pct=True)
        adf['aa_rank'] = adf.aa.rank(pct=True)

        pos.append(adf[adf.index == movieId])
        
    return pos

poses = all_positions(119145)

In [26]:
pd.concat(poses)

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba,ba_rank,aa_rank
movieId,,,,,,,,,
119145,1,3.500000,3.501384,10.366207,0.087980,3.500000,3.501262,0.584070,0.565714
119145,2,4.250000,3.501399,10.366310,0.161730,4.250000,3.622470,0.828054,0.910222
119145,3,3.833333,3.501394,10.366413,0.224443,3.833333,3.575895,0.749537,0.725422
119145,4,3.875000,3.501399,10.366516,0.278425,3.875000,3.605419,0.810880,0.740693
119145,5,3.900000,3.501404,10.366619,0.325381,3.900000,3.631099,0.837567,0.747378
119145,6,3.750000,3.501399,10.366722,0.366598,3.750000,3.592535,0.802653,0.690868
119145,7,3.785714,3.501404,10.366824,0.403067,3.785714,3.616000,0.821524,0.708968
119145,8,3.812500,3.501409,10.366927,0.435566,3.812500,3.636909,0.870321,0.717400
119145,9,3.833333,3.501414,10.367030,0.464707,3.833333,3.655659,0.884718,0.725422
